In [1]:
%reload_ext autoreload
%autoreload 2


from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pkg_resources
from symspellpy.symspellpy import SymSpell, Composition

from typing import Optional
from tqdm import tqdm

In [35]:
# Set max_dictionary_edit_distance to avoid spelling correction
spell = SymSpell(max_dictionary_edit_distance=4, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")

# term_index is the column of the term and count_index is the
# column of the term frequency
spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [40]:
def compare_seg(input: str, spell: SymSpell, max_edit_distance: int) -> Optional[tuple[Composition, Composition]]:
    with_spell = spell.word_segmentation(input, max_edit_distance=max_edit_distance)
    without_spell = spell.word_segmentation(input, max_edit_distance=0)
    if with_spell.corrected_string == without_spell.corrected_string:
        return None
    return (with_spell, without_spell)

In [68]:
compare_seg('miraclousdidit', spell, 2)

(Composition(segmented_string='miraclous did it', corrected_string='miraculous did it', distance_sum=3, log_prob_sum=-12.142232874753333),
 Composition(segmented_string='mira clo us did it', corrected_string='mira clo us did it', distance_sum=7, log_prob_sum=-28.825918894268174))

In [60]:
instant_domains = [x.strip() for x in open('instant-domains.csv').readlines()]
len(instant_domains)

1030

In [ ]:
comps = [compare_seg(i, spell, 2) for i in tqdm(instant_domains)]

In [ ]:
diffs = [c for c in comps if c is not None]
len(diffs)
diffs[0]

(Composition(segmented_string='fiverax', corrected_string='rivera', distance_sum=2, log_prob_sum=-5.665092898954126),
 Composition(segmented_string='fiver ax', corrected_string='fiver ax', distance_sum=3, log_prob_sum=-19.87040796373899))

In [ ]:
import csv
with open('diffs_2.csv', 'w') as f:
    wr = csv.DictWriter(f, fieldnames=['domain', 'corr', 'no_corr', 'corr_d', 'no_corr_d', 'corr_p', 'no_corr_p'])
    wr.writeheader()
    for w, wo in diffs:
        wr.writerow({
            'domain': f'{wo.corrected_string.replace(" ", "")}.com',
            'corr': w.corrected_string,
            'no_corr': wo.corrected_string,
            'corr_d': w.distance_sum,
            'corr_p': w.log_prob_sum,
            'no_corr_d': wo.distance_sum,
            'no_corr_p': wo.log_prob_sum}) 

In [88]:
import pandas as pd
pd.read_csv('diffs_2.csv').tail()

,domain,corr,no_corr,corr_d,no_corr_d,corr_p,no_corr_p
636,sydneybrachytherapy.com,sydney racy therapy,sydney brach y therapy,4,4,-15.476148,-28.076489
637,deepmindvsopenai.com,deep minds open,deep mind vs open a i,5,7,-12.945435,-29.631663
638,rilednradical.com,riled radical,riled n radical,2,3,-12.009431,-24.020116
639,reaphemp.com,reach emp,reap hemp,2,1,-10.252955,-11.425282
640,lonestarerides.com,lowest are rides,lone stare rides,3,2,-12.210575,-16.222795
